# 0 Imports

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import altair as alt

## 0.1 Loading Data

In [2]:
df4 = pd.read_csv('data/df4.csv')
ireland4 = pd.read_csv('data/ireland4.csv')

## 0.2 Manage Datasets

In [3]:
# Creating dataframe that holds all countries milk production
all_countries_milk = df4[df4['item_group'] == "Milk, Total"]
all_countries_milk = all_countries_milk.drop(columns=['item'])

# Creating dataframe that holds Ireland's milk production
eire_milk = ireland4[ireland4['item_group'] == "Milk, Total"]

# Creating dataframe that holds European countries milk production
europe_individual_milk = all_countries_milk[all_countries_milk['area_group'] == "Europe"]

# Creating dataframe that holds European countries milk production
europe_average_milk = all_countries_milk[all_countries_milk['area_group'] == "Europe"]
europe_average_milk = europe_average_milk.groupby(['area_group','year','item_group'],as_index=False).mean()

# Creating dataframe that holds worldwide countries milk production
all_countries_milk_trade_production_df = all_countries_milk

### 0.2.1 Top 20 Milk producing countries

In [4]:
all_countries_milk_sum = all_countries_milk.groupby(by=['area','item_group'], as_index=False).sum()
top20_milk_export_production = all_countries_milk_sum.sort_values(['export_quantity', 'export_value(1000US$)','production(tonnes)'],ascending=False).head(25)['area'].unique()
top20_milk_export_production

array(['Germany', 'France', 'Belgium',
       'United Kingdom of Great Britain and Northern Ireland',
       'Netherlands', 'Czechia', 'Austria', 'Belgium-Luxembourg',
       'Poland', 'Hungary', 'Slovenia', 'Denmark', 'Portugal', 'Latvia',
       'Belarus', 'Luxembourg', 'Spain', 'Ireland', 'Australia',
       'Slovakia', 'New Zealand', 'Estonia', 'United States of America',
       'Uruguay', 'Lithuania'], dtype=object)

In [5]:
top_20_country_pattern = '|'.join(top20_milk_export_production)

top20_countries_milk_trade_production_df = all_countries_milk[all_countries_milk['area'].str.match(top_20_country_pattern)]
top20_countries_milk_trade_production_df.head()

,Unnamed: 0,area,area_group,year,item_group,area_harvested(ha),laying(1000head),milkanimals(head),prod_popultn,producing_animals/slaughtered(head),production(tonnes),stocks(head),yield(hg/ha),yield/carcass_weight(hg/an),export_quantity,export_value(1000US$),import_quantity,import_value(1000US$),net_export,net_export_value(1000US$)
27963,27963,Australia,Oceania,1961,"Milk, Total",0.0,0.0,3162000.0,0.0,0.0,6277000.0,0.0,19851.0,0.0,4500.0,575.0,0.0,0.0,4500.0,575.0
28079,28079,Australia,Oceania,1962,"Milk, Total",0.0,0.0,3230000.0,0.0,0.0,6765000.0,0.0,20944.0,0.0,6500.0,810.0,0.0,0.0,6500.0,810.0
28195,28195,Australia,Oceania,1963,"Milk, Total",0.0,0.0,3263000.0,0.0,0.0,6879000.0,0.0,21082.0,0.0,10000.0,1245.0,0.0,0.0,10000.0,1245.0
28311,28311,Australia,Oceania,1964,"Milk, Total",0.0,0.0,3078000.0,0.0,0.0,7014000.0,0.0,22788.0,0.0,1630.0,353.0,46.0,20.0,1584.0,333.0
28427,28427,Australia,Oceania,1965,"Milk, Total",0.0,0.0,3012000.0,0.0,0.0,7128000.0,0.0,23665.0,0.0,1266.0,150.0,154.0,60.0,1112.0,90.0


 ### 0.2.2 All countries name updates
 **Step intended to rename countries**

In [6]:
country_codes_df = pd.read_csv('data/FAOSTAT_data_12-5-2022.csv', encoding='latin-1')
country_codes_df = country_codes_df[['Country','ISO3 Code']]
country_codes_df.head()

,Country,ISO3 Code
0,Algeria,DZA
1,Angola,AGO
2,Benin,BEN
3,Botswana,BWA
4,Burkina Faso,BFA


In [7]:
# rename countries with the name we have in our dataset all_countries_milk_trade_production_df
country_codes_df.loc[country_codes_df['Country'] == 'Bahamas (the)', 'Country'] = 'Bahamas'
country_codes_df.loc[country_codes_df['Country'] == 'Central African Republic (the)', 'Country'] = 'Central African Republic'
country_codes_df.loc[country_codes_df['Country'] == 'Hong Kong', 'Country'] = 'China, Hong Kong SAR'
country_codes_df.loc[country_codes_df['Country'] == 'Taiwan (Province of China)', 'Country'] = 'China, Taiwan Province of'
country_codes_df.loc[country_codes_df['Country'] == 'Comoros (the)', 'Country'] = 'Comoros'
country_codes_df.loc[country_codes_df['Country'] == 'Congo (the)', 'Country'] = 'Congo'
country_codes_df.loc[country_codes_df['Country'] == 'Congo (the Democratic Republic of the)', 'Country'] = 'Democratic Republic of the Congo'
country_codes_df.loc[country_codes_df['Country'] == 'Dominican Republic (the)', 'Country'] = 'Dominican Republic'
country_codes_df.loc[country_codes_df['Country'] == 'Gambia (the)', 'Country'] = 'Gambia'
country_codes_df.loc[country_codes_df['Country'] == "Lao People's Democratic Republic (the)", 'Country'] = "Lao People's Democratic Republic"
country_codes_df.loc[country_codes_df['Country'] == 'Netherlands (the)', 'Country'] = 'Netherlands'
country_codes_df.loc[country_codes_df['Country'] == 'Niger (the)', 'Country'] = 'Niger'
country_codes_df.loc[country_codes_df['Country'] == 'Republic of North Macedonia', 'Country'] = 'North Macedonia'
country_codes_df.loc[country_codes_df['Country'] == 'Palestine, State of'] = 'Palestine'
country_codes_df.loc[country_codes_df['Country'] == 'Philippines (the)', 'Country'] = 'Philippines',
country_codes_df.loc[country_codes_df['Country'] == 'Korea (the Republic of)', 'Country'] = 'Republic of Korea'
country_codes_df.loc[country_codes_df['Country'] == "Korea (the Democratic People's Republic of)", 'Country'] = "Democratic People's Republic of Korea"
country_codes_df.loc[country_codes_df['Country'] == 'Moldova (the Republic of)', 'Country'] = 'Republic of Moldova'
country_codes_df.loc[country_codes_df['Country'] == 'Russian Federation (the)', 'Country'] = 'Russian Federation'
country_codes_df.loc[country_codes_df['Country'] == 'Sudan (the)', 'Country'] = 'Sudan'
country_codes_df.loc[country_codes_df['Country'] == 'United Arab Emirates (the)', 'Country'] = 'United Arab Emirates'
country_codes_df.loc[country_codes_df['Country'] == 'United Kingdom of Great Britain and Northern Ireland (the)', 'Country'] = 'United Kingdom of Great Britain and Northern Ireland'
country_codes_df.loc[country_codes_df['Country'] == 'Tanzania, United Republic of', 'Country'] = 'United Republic of Tanzania'
country_codes_df.loc[country_codes_df['Country'] == 'United States of America (the)', 'Country'] = 'United States of America'

In [8]:
# Rename countries whose name have been modified suring history like...
# Czechoslovakia existed up to 1992, it needs to be renamed to Czechia  
# USSR disolved in 1991, the main map will be using the name Russian Federation

all_countries_milk_trade_production_df.loc[all_countries_milk_trade_production_df['area'] == 'Ethiopia PDR', 'area'] = 'Ethiopia'
all_countries_milk_trade_production_df.loc[all_countries_milk_trade_production_df['area'] == 'Czechoslovakia', 'area'] = 'Czechia' 
all_countries_milk_trade_production_df.loc[all_countries_milk_trade_production_df['area'] == 'USSR', 'area'] = 'Russian Federation'
all_countries_milk_trade_production_df.loc[all_countries_milk_trade_production_df['area'] == 'Yugoslav SFR', 'area'] = 'Serbia'
all_countries_milk_trade_production_df.loc[all_countries_milk_trade_production_df['area'] == 'Serbia and Montenegro', 'area'] = 'Serbia'
all_countries_milk_trade_production_df.loc[all_countries_milk_trade_production_df['area'] == 'Sudan (former)', 'area'] = 'Sudan'


In [9]:
# rename the Country and iso3 code alpha - 3 code to merge the country codes dataframe VS all_countries_milk_trade_production_df
country_codes_df = country_codes_df.rename(columns={"Country": "area"})
country_codes_df = country_codes_df.rename(columns={"ISO3 Code": "code"})

In [10]:
# merge  countries codes and datafram to join the codes according the area to display the main map

df1 = all_countries_milk_trade_production_df
df2 = country_codes_df

All_countries_milk_trade_production_codes_df = pd.merge(df1, 
                      df2, 
                      on ='area', 
                      how ='left')

In [11]:
All_countries_milk_trade_production_codes_df.head()

,Unnamed: 0,area,area_group,year,item_group,area_harvested(ha),laying(1000head),milkanimals(head),prod_popultn,producing_animals/slaughtered(head),...,stocks(head),yield(hg/ha),yield/carcass_weight(hg/an),export_quantity,export_value(1000US$),import_quantity,import_value(1000US$),net_export,net_export_value(1000US$),code
0,29,Afghanistan,Asia,1961,"Milk, Total",0.0,0.0,700000.0,0.0,0.0,...,0.0,5000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AFG
1,66,Afghanistan,Asia,1962,"Milk, Total",0.0,0.0,700000.0,0.0,0.0,...,0.0,5000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AFG
2,103,Afghanistan,Asia,1963,"Milk, Total",0.0,0.0,780000.0,0.0,0.0,...,0.0,5128.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AFG
3,140,Afghanistan,Asia,1964,"Milk, Total",0.0,0.0,780000.0,0.0,0.0,...,0.0,5128.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AFG
4,177,Afghanistan,Asia,1965,"Milk, Total",0.0,0.0,870000.0,0.0,0.0,...,0.0,5172.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AFG


# 1. Dashboard creation

## 1.1 Dash instance to initialize

In [12]:
# create an instance of the Dash class to initialize the app

external_stylesheets = [
    {
        "href": "https://fonts.googleapis.com/css2?"
                "family=Lato:wght@400;700&display=swap",
        "rel": "stylesheet",
    },
]
app = Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "Interactive Dashboard for Analytics: Milk production!"

## 1.2 App Layout

In [13]:
app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.P(children="🥛", className="header-emoji"),
                html.H1(
                    children="Milk Production", className="header-title"
                ),
                html.P(
                    children="Analysis of Milk & trade production Ireland vs Top 25 Producers in the World",
                    style={'text-align': 'center'},
                ),
            ],
            className="header",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(children="Country to Compare", className="menu-title"),
                        dcc.Dropdown(
                            id="area-filter",
                            options=[
                                {"label": area, "value": area}
                                for area in np.sort(top20_countries_milk_trade_production_df.area.unique())
                            ],
                            value="Portugal",
                            clearable=False,
                            className="dropdown",
                        ),
                    ]
                ),
                html.Div(
                    children=[
                        html.Div(children="Item in Analysis", className="menu-title"),
                        dcc.Dropdown(
                            id="item-filter",
                            options=[
                                {"label": item_group, "value": item_group}
                                for item_group in top20_countries_milk_trade_production_df.item_group.unique()
                            ],
                            value="Milk, Total",
                            clearable=False,
                            searchable=False,
                            className="dropdown",
                        ),
                    ],
                ),
            ],
            className="menu",
        ),
    
    html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="price-chart_ire", figure ={}, config={"autosizable":True, "displayModeBar": False},
                    ),
                    style={'padding': 10, 'flex': 1}
                    #className="card",
                ),
                html.Div(
                    children=dcc.Graph(
                        id="price-chart_top25", figure ={}, config={"autosizable":True, "displayModeBar": False},
                    ),
                    style={'padding': 10, 'flex': 1}
                    #className="card",
                ),
            ],
            style={'display': 'flex', 'flex-direction': 'row'}
        #className="menu",
        ),
    html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="volume-chart_ire", figure ={}, config={"autosizable":True, "displayModeBar": False},
                    ),
                    style={'padding': 10, 'flex': 1}
                    #className="card",
                ),
                html.Div(
                    children=dcc.Graph(
                        id="volume-chart_top25", figure ={}, config={"autosizable":True, "displayModeBar": False},
                    ),
                    style={'padding': 10, 'flex': 1}
                    #className="card",
                ),
            ],
            style={'display': 'flex', 'flex-direction': 'row'}
            #className="menu",
        ),
    html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="volume-barchart_ire", figure ={}, config={"autosizable":True, "displayModeBar": False},
                    ),
                    style={'padding': 10, 'flex': 1}
                    #className="card",
                ),
                html.Div(
                    children=dcc.Graph(
                        id="volume-barchart_top25", figure ={}, config={"autosizable":True, "displayModeBar": False},
                    ),
                    style={'padding': 10, 'flex': 1}
                    #className="card",
                ),
            ],
            style={'display': 'flex', 'flex-direction': 'row'}
            #className="menu",
        ),
        
    html.Br(),
    html.Label('Select the Year to see Worldwide milk production'),    
    dcc.Slider(
            id='slct_year',
            min=1961,
            max=2020,
            step=1,
            value=1961,
            tooltip={"placement": "bottom", "always_visible": True},
            ),
    html.Div(id='output_container', children=[]),
    html.Br(),
    dcc.Graph(id='my_map', figure={})           
    ]
)

## 1.3 App Call back
**Step done for plots/map and reaction to user imputs - like filtering**

### 1.3.1 Graphs

In [14]:
@app.callback(
    [Output("price-chart_ire", 'figure'),Output("price-chart_top25", 'figure'),
     Output("volume-chart_ire", 'figure'),Output("volume-chart_top25", 'figure'),
     Output("volume-barchart_ire", 'figure'),Output("volume-barchart_top25", 'figure')],
    [
        Input("area-filter", "value"),
        Input("item-filter", "value"),
    ],
)
def update_charts(area, item_group):
    mask = ((All_countries_milk_trade_production_codes_df.area == area)
            & (All_countries_milk_trade_production_codes_df.item_group == item_group)
        )
    filtered_data = All_countries_milk_trade_production_codes_df.loc[mask, :]
    df_ireland = eire_milk[eire_milk['item_group'] == item_group]
    
    price_chart_ire = {
        "data": [
            {
            "x": df_ireland["year"],
            "y": df_ireland["production(tonnes)"],
            "type": "bar",
            "hovertemplate": "$%{y:.2f}<extra></extra>",
        },
    ],
        "layout": {
            "title": {
                "text": "Production (tonnes) of Milk in Ireland",
                "x": 0.05,
                "xanchor": "left",
            },
            "xaxis": {"fixedrange": True},
            "yaxis": {"fixedrange": True},
            "colorway": ["#17B897"],
        },
    }
    
    price_chart_top25 = {
        "data": [
            {
            "x": filtered_data["year"],
            "y": filtered_data["production(tonnes)"],
            "type": "bar",
            "hovertemplate": "$%{y:.2f}<extra></extra>",
        },
    ],
        "layout": {
            "title": {
                "text": "Production (tonnes) of Milk in selected country",
                "x": 0.05,
                "xanchor": "left",
            },
            "xaxis": {"fixedrange": True},
            "yaxis": {"fixedrange": True},
            "colorway": ["#E12D39"],
        },
    }

    volume_chart_ire = {
        "data": [
            {
                "x": df_ireland["year"],
                "y": df_ireland["import_value(1000US$)"],
                "type": "lines",
            },
        ],
        "layout": {
            "title": {"text": "Import Value milk in Ireland (USD)", "x": 0.05, "xanchor": "left"},
            "xaxis": {"fixedrange": True},
            "yaxis": {"tickprefix": "$", "fixedrange": True},
            "colorway": ["#17B897"],
        },
    }
    
    volume_chart_top25 = {
        "data": [
            {
                "x": filtered_data["year"],
                "y": filtered_data["import_value(1000US$)"],
                "type": "lines",
            },
        ],
        "layout": {
            "title": {"text": "Import Value milk in selected country (USD)", "x": 0.05, "xanchor": "left"},
            "xaxis": {"fixedrange": True},
            "yaxis": {"tickprefix": "$", "fixedrange": True},
            "colorway": ["#E12D39"],
        },
    }
    
    volume_barchart_ire = {
        "data": [
            {
                "x": df_ireland["year"],
                "y": df_ireland["export_value(1000US$)"],
                "type": "lines",
            },
        ],
        "layout": {
            "title": {"text": "Export Value milk in Ireland (USD)", "x": 0.05, "xanchor": "left"},
            "xaxis": {"fixedrange": True},
            "yaxis": {"tickprefix": "$", "fixedrange": True},
            "colorway": ["#17B897"],
        },
    }
    
    volume_barchart_top25 = {
        "data": [
            {
                "x": filtered_data["year"],
                "y": filtered_data["export_value(1000US$)"],
                "type": "lines",
            },
        ],
        "layout": {
            "title": {"text": "Export Value milk in selected country (USD)", "x": 0.05, "xanchor": "left"},
            "xaxis": {"fixedrange": True},
            "yaxis": {"tickprefix": "$", "fixedrange": True},
            "colorway": ["#E12D39"],
        },
    }
   

    return price_chart_ire, price_chart_top25, volume_chart_ire, volume_chart_top25, volume_barchart_ire, volume_barchart_top25


### 1.3.2 Map

In [15]:
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_map', component_property='figure')],
    [Input(component_id='slct_year', component_property='value'),
     Input(component_id='item-filter', component_property='value')]
)
def update_graph(option_slctd,item):
    print(option_slctd)
    print(type(option_slctd))

    container = "The year chosen is: {} | The item is: {}".format(option_slctd,item) 

    dff = All_countries_milk_trade_production_codes_df.copy()
    dff = dff[dff["year"] == option_slctd]
    dff = dff[dff['item_group'].str.contains(item)]

    # Plotly Express
    fig = px.choropleth(
        data_frame=dff,
        locations='code',
        color='production(tonnes)',
        hover_data = ['milkanimals(head)','production(tonnes)','yield(hg/ha)','export_quantity','export_value(1000US$)','import_quantity','import_value(1000US$)'],
        color_continuous_scale=px.colors.sequential.YlOrRd,
        animation_frame="year",
    )
    
    fig.update_layout(
        title_text = 'World Map',
        # set projection style for the plot
        geo = dict(showframe = True, showocean = True,showlakes = True,showcoastlines = True, projection = dict(type = 'hammer')),

    )
    return container, fig

## 1.4 Final Output

In [ ]:
if __name__ == "__main__":
    app.run_server(debug=False)